In [1]:
from fastai.vision.all import *
import torchaudio
import librosa
from nnAudio.features.cqt import CQT1992v2
from scipy.signal import tukey


C:\Users\anaconda\envs\Pytorch_19\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class CFG:
    apex=False
    debug=False
    print_freq=100
    num_workers=4
    model_name='tf_efficientnet_b7_ns'
    scheduler='CosineAnnealingLR' # ['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts']
    epochs=3
    #factor=0.2 # ReduceLROnPlateau
    #patience=4 # ReduceLROnPlateau
    #eps=1e-6 # ReduceLROnPlateau
    T_max=3 # CosineAnnealingLR
    #T_0=3 # CosineAnnealingWarmRestarts
    lr=1e-4
    min_lr=1e-6
    batch_size=48
    weight_decay=1e-6
    gradient_accumulation_steps=1
    max_grad_norm=1000
    qtransform_params={"hop_length": 1024, "n_bins": 64,'fmax':None}    
    seed=42
    target_size=1
    target_col='target'
    n_fold=5
    trn_fold=[0] # [0, 1, 2, 3, 4]
    train=True
    grad_cam=False


In [3]:
df_train = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification/train.csv')
df_test = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification/test.csv')
submission = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification/sample_submission.csv')

In [4]:
train_path = Path('C:/Users/Kaggle/Pog_Music_Classification/train/')
test_path = Path('C:/Users/Kaggle/Pog_Music_Classification/test/')

In [5]:
print(df_train.shape,df_test.shape)

(19922, 5) (5078, 3)


In [6]:
df_train.head()

,song_id,filename,filepath,genre_id,genre
0,10150,010150.ogg,train/010150.ogg,7,Instrumental
1,7358,007358.ogg,train/007358.ogg,2,Punk
2,20573,020573.ogg,train/020573.ogg,5,Folk
3,11170,011170.ogg,train/011170.ogg,12,Old-Time / Historic
4,16662,016662.ogg,train/016662.ogg,1,Rock


In [7]:
import glob
valid_filenames = []
for i in glob.glob(r'C:/Users/Kaggle/Pog_Music_Classification/resampled_16k/train/*.npy'):
    fnames = (i.split('\\')[1])
    valid_filenames.append(fnames.split('.')[0]+'.ogg')

In [8]:
import glob
valid_filenames_test = []
for i in glob.glob(r'C:/Users/Kaggle/Pog_Music_Classification/resampled_16k/test/*.npy'):
    fnames = (i.split('\\')[1])
    valid_filenames_test.append(fnames.split('.')[0]+'.ogg')

In [9]:
df_train = df_train[df_train['filename'].isin(valid_filenames)].reset_index(drop=True)
df_test_valid = df_test[df_test['filename'].isin(valid_filenames_test)].reset_index(drop=True)

In [10]:
### Some files giving exception in cqt
df_train = df_train[df_train['song_id']!=17400].reset_index(drop=True)

In [11]:
print(df_train.shape,df_test.shape)

(19908, 5) (5078, 3)


### Upsample minority classes

In [12]:
df_train['genre_id'].value_counts()

1     3095
0     3071
2     2582
3     1800
4     1756
5     1214
6     1181
7     1044
8      945
9      814
10     796
11     495
12     408
13     306
14     142
15      94
16      94
17      58
18      13
Name: genre_id, dtype: int64

In [13]:
ExtremeMinority = [18]
SomeMinority = [17]
LowMinority = [14,15,16]

# df_train = pd.concat([df_train,df_train[df_train['genre_id'].isin(ExtremeMinority)].sample(n=37,replace=True,random_state=42)],0).reset_index(drop=True)
# df_train = pd.concat([df_train,df_train[df_train['genre_id'].isin(SomeMinority)].sample(n=42,replace=True,random_state=42)],0).reset_index(drop=True)
# df_train = pd.concat([df_train,df_train[df_train['genre_id'].isin(LowMinority)].sample(n=50,replace=True,random_state=42)],0).reset_index(drop=True)

In [14]:
df_train.tail()#,df_train.shape

,song_id,filename,filepath,genre_id,genre
19903,9337,009337.ogg,train/009337.ogg,0,Electronic
19904,8340,008340.ogg,train/008340.ogg,4,Hip-Hop
19905,16248,016248.ogg,train/016248.ogg,4,Hip-Hop
19906,11875,011875.ogg,train/011875.ogg,2,Punk
19907,6579,006579.ogg,train/006579.ogg,1,Rock


In [15]:
df_train['genre_id'].value_counts()

1     3095
0     3071
2     2582
3     1800
4     1756
5     1214
6     1181
7     1044
8      945
9      814
10     796
11     495
12     408
13     306
14     142
15      94
16      94
17      58
18      13
Name: genre_id, dtype: int64

#### condense bottom 8 classes into single class

In [16]:
df_train['genre_id'] = df_train['genre_id'].apply(lambda x: 5 if x>=5 else x).values

In [17]:
df_train['genre_id'].value_counts()

5    7604
1    3095
0    3071
2    2582
3    1800
4    1756
Name: genre_id, dtype: int64

### Create Folds

In [18]:
df_train['fold'] = 0

In [19]:
from sklearn.model_selection import StratifiedKFold
N_folds = 5
kf = StratifiedKFold(5,random_state=42,shuffle=True)
fld = 0
for i,(_,val_id) in enumerate(kf.split(df_train['filename'],df_train['genre_id'])):
    df_train.loc[val_id,'fold'] = int(fld)
    fld += 1

In [20]:
df_train.head()

,song_id,filename,filepath,genre_id,genre,fold
0,10150,010150.ogg,train/010150.ogg,5,Instrumental,1
1,7358,007358.ogg,train/007358.ogg,2,Punk,3
2,20573,020573.ogg,train/020573.ogg,5,Folk,1
3,11170,011170.ogg,train/011170.ogg,5,Old-Time / Historic,4
4,16662,016662.ogg,train/016662.ogg,1,Rock,0


In [21]:
df_test.head()

,song_id,filename,filepath
0,7072,007072.ogg,test/007072.ogg
1,10207,010207.ogg,test/010207.ogg
2,20008,020008.ogg,test/020008.ogg
3,10924,010924.ogg,test/010924.ogg
4,21896,021896.ogg,test/021896.ogg


#### Channel wise max: 2.7517, 2.7950

#### Dataset

In [22]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision.transforms import Compose
import torchvision

tfms = Compose(
#     [torchvision.transforms.Resize((128,128))])
    [torchvision.transforms.CenterCrop((128,960))])


In [23]:
import torch_audiomentations as tA

def get_transforms(*, data):
    
    if data == 'train':
        return tA.Compose(
                transforms=[
#                      tA.ShuffleChannels(p=0.1,mode="per_example",p_mode="per_example", sample_rate= 16000),
                    tA.PeakNormalization(p=0.5,apply_to='only_too_loud_sounds',mode="per_example",p_mode="per_example",sample_rate=16000),
#                     tA.BandPassFilter(),
                    tA.TimeInversion(p=0.5,p_mode="per_example",sample_rate=16000),
#                     tA.PitchShift()
#                      tA.AddColoredNoise(p=0.1,mode="per_channel",p_mode="per_channel", sample_rate=16000,max_snr_in_db = 15),
                     tA.Shift(p=0.5,mode="per_example",p_mode="per_example", sample_rate=16000,
                              max_shift=0.025, min_shift=-0.025),
                ])

    elif data == 'valid':
        return tA.Compose([
        ])

In [24]:
class POGFiles(torch.utils.data.Dataset):
    
    def __init__(self, df, 
                 train = True,
                 augmentations = None,
                 normalize=False,
                 ch0Max=2.7517,
                 ch1Max=2.7950,
                 HOP_LEN = 1024,
                 N_FFT=1024,
                 n_mels=224,
                 transform=tfms,
                 audiotfms=None,
                 applyTukey=False,
                 quantizeData=False,
                 MakeThreeChannel= False,
                 bins_per_octave = 24,
                 n_bins = 64
                ):
        
        
        self.df = df
        self.train = train
        self.ch0Max = ch0Max
        self.ch1Max = ch1Max        
        self.N_FFT = N_FFT
        self.n_bins = n_bins
        self.bins_per_octave = bins_per_octave
        self.HOP_LEN = HOP_LEN
        self.transform = transform
        self.n_mels= n_mels
        self.audiotfms = audiotfms
        self.normalize = normalize
        self.MakeThreeChannel = MakeThreeChannel
        
    def __len__(self):
        return len(self.df)
    
    def load_file(self, filename):
        if self.train:
            spec = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/spec/512_128/train/{filename}')
        else:
            spec = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/spec/512_128/test/{filename}')
            
        return spec


    def __getitem__(self, index):
        row  = self.df.iloc[index]
        
        samples = self.load_file(row.filename.split('.')[0]+'.npy')
        samples = torch.from_numpy(samples).float()
        
        if self.transform is not None:
            samples = self.transform(samples)
            
        if self.train:
            label = torch.tensor(row.genre_id,dtype=torch.long)
            return samples, label
        else:

            return samples

In [ ]:
# a,b = next(iter(POGFiles(df_train)))
# a.shape

In [25]:
a,b = next(iter(POGFiles(df_train,transform=None)))
#` a,b = next(iter(POGFiles(df_train,transform=None,n_mels=128,HOP_LEN=576,N_FFT=512)))
a.shape

torch.Size([1, 128, 938])

In [26]:
def gem_1d_res(x, p=3, eps=1e-6):
    return F.avg_pool1d(x.clamp(min=eps).pow(p), 2,2).pow(1./p)

def gem_1d(x, p=3, eps=1e-6):
    return F.avg_pool1d(x.clamp(min=eps).pow(p), (x.size(-1),)).pow(1./p )

#### FastAI model

In [27]:
df_train['genre_id'].nunique()

6

In [28]:
df_train.head()

,song_id,filename,filepath,genre_id,genre,fold
0,10150,010150.ogg,train/010150.ogg,5,Instrumental,1
1,7358,007358.ogg,train/007358.ogg,2,Punk,3
2,20573,020573.ogg,train/020573.ogg,5,Folk,1
3,11170,011170.ogg,train/011170.ogg,5,Old-Time / Historic,4
4,16662,016662.ogg,train/016662.ogg,1,Rock,0


In [29]:
import timm
timm.list_models()

['adv_inception_v3',
 'bat_resnext26ts',
 'beit_base_patch16_224',
 'beit_base_patch16_224_in22k',
 'beit_base_patch16_384',
 'beit_large_patch16_224',
 'beit_large_patch16_224_in22k',
 'beit_large_patch16_384',
 'beit_large_patch16_512',
 'botnet26t_256',
 'botnet50ts_256',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_tiny',
 'convit_base',
 'convit_small',
 'convit_tiny',
 'convmixer_768_32',
 'convmixer_1024_20_ks9_p14',
 'convmixer_1536_20',
 'convnext_base',
 'convnext_base_384_in22ft1k',
 'convnext_base_in22ft1k',
 'convnext_base_in22k',
 'convnext_large',
 'convnext_large_384_in22ft1k',
 'convnext_large_in22ft1k',
 'convnext_large_in22k',
 'convnext_small',
 'convnext_tiny',
 'convnext_tiny_hnf',
 'convnext_xlarge_384_in22ft1k',
 'convnext_xlarge_in22ft1k',
 'convnext_x

In [30]:
# timm.create_model('resnest50d',in_chans=1)

In [31]:
# timm.create_model('resnext50_32x4d',in_chans=1).conv1.stride

In [32]:
df_train['genre_id'].value_counts()

5    7604
1    3095
0    3071
2    2582
3    1800
4    1756
Name: genre_id, dtype: int64

In [33]:
df_train.head()

,song_id,filename,filepath,genre_id,genre,fold
0,10150,010150.ogg,train/010150.ogg,5,Instrumental,1
1,7358,007358.ogg,train/007358.ogg,2,Punk,3
2,20573,020573.ogg,train/020573.ogg,5,Folk,1
3,11170,011170.ogg,train/011170.ogg,5,Old-Time / Historic,4
4,16662,016662.ogg,train/016662.ogg,1,Rock,0


In [35]:
NCLASS = df_train['genre_id'].nunique()

In [36]:
def run():
    oof = df_train.copy()
    oversampleTrain = False
    
    for fold_num in [0,1,2,3,4]:
        print('*****************************************')
        print(f'Training Fold {fold_num}')
        print('*****************************************')

    #with IPyExperimentsPytorch() as exp:
        kernel_type = 'resnest50d_MonoSpec_Hop512_Mels128_6Cls'
        kernel_type_prev = 'resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128'
        OUTPUT_DIR = f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/'
        if not os.path.exists(OUTPUT_DIR):
            os.makedirs(OUTPUT_DIR)

            
        crit = CrossEntropyLossFlat() #LabelSmoothingCrossEntropy(0.025) #CrossEntropyLossFlat() #FocalLoss()
        batch_size = 32
        n_epochs = 7

        training_fold = df_train.query(f'fold!={fold_num}').reset_index(drop=True, inplace=False)
        
        if oversampleTrain:
            print('--------- Oversampling training dataset -----------')
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(ExtremeMinority)].sample(n=150,replace=True,random_state=42)],0).reset_index(drop=True)
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(SomeMinority)].sample(n=150,replace=True,random_state=42)],0).reset_index(drop=True)
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(LowMinority)].sample(n=100,replace=True,random_state=42)],0).reset_index(drop=True)
                
        train_ds = POGFiles(training_fold,
                            transform=tfms,
                            n_mels=128,HOP_LEN=512,N_FFT=512
                           )

        validation_fold = df_train.query(f'fold=={fold_num}').reset_index(drop=True, inplace=False)
        valid_ds = POGFiles(validation_fold,
                            transform=tfms,
                            n_mels=128,HOP_LEN=512,N_FFT=512
                            )

        print(f'- Training samples: {len(train_ds)}\n- Validation Samples : {len(valid_ds)}')

        bs = batch_size
        train_dl = torch.utils.data.DataLoader(train_ds, batch_size=bs, num_workers=0,pin_memory=False)
        valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=bs*2, num_workers=0,shuffle=False,pin_memory=False)

        dls = DataLoaders(train_dl, valid_dl)
        
        
        model = timm.create_model('resnest50d', pretrained=False, in_chans=1)#get_model(resnet34)
        model.conv1[0].stride = (1,1)
        
#         model.conv1[0].stride = (1,1)
#         model.conv_stem.stride = (1,1)
#         model.stem[0].stride = (2,2)
        model.fc = nn.Linear(2048,NCLASS)

    
        f1_score = F1Score(average="micro")
        
        
        
        ### MixUp() in callbacks
#        ReduceLROnPlateau(monitor='valid_loss', min_delta=0.1,patience=2,min_lr=1e-7)
# GradientAccumulation (n_acc = 2)
        
        learn = Learner(dls, model,opt_func=ranger,loss_func=crit,metrics=[f1_score],cbs=[MixUp(alpha=0.4),
                                                                                          SaveModelCallback('f1_score', every_epoch=True),
                                                                                          CSVLogger(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/{fold_num}logs.csv')]).to_fp16()
        
#         learn.load()
#         weights = torch.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type_prev}/fold_{fold_num}.pth')['model']
#         learn.model.load_state_dict(weights,strict=False)

        learn.fit_one_cycle(n_epochs, 1e-4, wd=1e-03)
#         learn.fit_flat_cos(n_epochs, 1e-4, wd=1e-03)
#         learn = learn.to_fp16()
        learn.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}')
        learn = learn.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}')
        
        learn.model.eval()
        test_df = df_test
        test_ds = POGFiles(df_test_valid,
                           transform=tfms,
                           n_mels=128,HOP_LEN=512,N_FFT=512,
                           train=False
                           )

        test_ds.input_path = Path(test_path)

        bs = batch_size
        test_dl = torch.utils.data.DataLoader(test_ds, batch_size=bs*2, num_workers=0, shuffle=False)
        
        preds = []
        probs = []
        for xb  in progress_bar(test_dl):
            with torch.no_grad():output = learn.model(xb.cuda())
            probs.append(torch.softmax(output.float(),1).squeeze().cpu())
            preds.append(torch.argmax(output.float(),1).squeeze().cpu())
        
        oof_ = []
        for xb,_  in progress_bar(valid_dl):
            with torch.no_grad():output = learn.model(xb.cuda())
            oof_.append(torch.softmax(output.float(),1).squeeze().cpu())
        
        oof_ = [t.detach().numpy() for t in oof_]
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/test_fold_{fold_num}_probs.npy',np.array(probs))
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',oof_)
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',np.array(validation_fold.song_id))        

        preds = torch.cat(preds)
        sample_df = df_test_valid.copy()
        sample_df['target'] = preds
        sample_df.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}.csv', index=False)

In [37]:
if __name__=='__main__':
    run()

*****************************************
Training Fold 0
*****************************************
- Training samples: 15926
- Validation Samples : 3982


epoch,train_loss,valid_loss,f1_score,time
0,1.389881,1.272495,0.494977,17:31
1,1.307333,1.211703,0.513561,18:30
2,1.251487,1.138124,0.534405,14:12
3,1.230622,1.096310,0.551984,13:58
4,1.208820,1.032013,0.591411,15:50
5,1.185399,1.020152,0.598694,15:18
6,1.176112,1.013033,0.602712,14:17


*****************************************
Training Fold 1
*****************************************
- Training samples: 15926
- Validation Samples : 3982


epoch,train_loss,valid_loss,f1_score,time
0,1.411099,1.301286,0.478403,15:28
1,1.308183,1.173180,0.531642,13:59
2,1.279520,1.121595,0.542190,13:57
3,1.234973,1.091733,0.559267,13:57
4,1.206926,1.038347,0.590909,13:58
5,1.191163,1.025938,0.593169,13:59
6,1.184545,1.021661,0.597438,13:58


*****************************************
Training Fold 2
*****************************************
- Training samples: 15926
- Validation Samples : 3982


epoch,train_loss,valid_loss,f1_score,time
0,1.396679,1.276700,0.478905,13:57
1,1.314504,1.202716,0.503265,13:57
2,1.269041,1.149488,0.536916,13:57
3,1.240804,1.094165,0.562783,13:57
4,1.196826,1.057869,0.569312,13:57
5,1.187822,1.025918,0.590407,13:57
6,1.180885,1.018786,0.595681,13:57


*****************************************
Training Fold 3
*****************************************
- Training samples: 15927
- Validation Samples : 3981


epoch,train_loss,valid_loss,f1_score,time
0,1.393466,1.276394,0.499372,14:07
1,1.312418,1.168225,0.527254,13:57
2,1.279034,1.135892,0.537805,13:57
3,1.228889,1.102201,0.555388,13:57
4,1.211996,1.035507,0.585531,13:57
5,1.187854,1.008838,0.596333,13:57
6,1.191477,1.009986,0.594825,13:57


*****************************************
Training Fold 4
*****************************************
- Training samples: 15927
- Validation Samples : 3981


epoch,train_loss,valid_loss,f1_score,time
0,1.402167,1.300265,0.474504,13:58
1,1.312249,1.203780,0.513439,13:57
2,1.272490,1.124104,0.546596,16:24
3,1.230068,1.084116,0.561919,14:13
4,1.200481,1.055364,0.572972,13:57
5,1.179469,1.020999,0.588797,13:57
6,1.183386,1.017413,0.590806,13:57


In [ ]:
submission

### OOF

In [38]:
OOFOut = pd.DataFrame()
for fld in range(5):
    fold_num = fld
    kernel_type= 'resnest50d_MonoSpec_Hop512_Mels128_6Cls'
    OOF = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',allow_pickle=True)
    ID = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',allow_pickle=True)
    OOF = ([item for sublist in OOF for item in sublist])
    OOF = pd.DataFrame(OOF)
    OOF['ID'] = ID
    OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)


In [39]:
OOFOut['ID']

0        11170
1        12181
2        17311
3        16076
4         4764
         ...  
19903    12998
19904     8377
19905     2729
19906    22703
19907    16248
Name: ID, Length: 19908, dtype: int64

In [40]:
OOFOut.tail()

,0,1,2,3,4,5,ID
19903,0.130617,0.095281,0.063305,0.244168,0.175632,0.290997,12998
19904,0.597983,0.070345,0.031782,0.018322,0.021660,0.259908,8377
19905,0.056062,0.243812,0.200636,0.054803,0.048698,0.395989,2729
19906,0.081086,0.394845,0.264938,0.059856,0.013147,0.186128,22703
19907,0.018628,0.014854,0.047579,0.011734,0.814181,0.093024,16248


In [41]:
OOFOut = pd.merge(OOFOut,df_train[['song_id','genre_id']],left_on='ID',right_on='song_id',how='left')

In [49]:
OOFOut['genre_id'].max()

5

In [47]:
OOF_preds=OOFOut.iloc[:,:-3].idxmax(1)


In [50]:
OOFOut[~OOFOut['genre_id']

,0,1,2,3,4,5,ID,song_id,genre_id
0,0.048578,0.032437,0.007142,0.044184,0.001151,0.866508,11170,11170,5
1,0.507800,0.023691,0.007191,0.026926,0.056082,0.378311,12181,12181,0
2,0.076020,0.074538,0.026568,0.061762,0.015721,0.745391,17311,17311,5
3,0.530656,0.015973,0.003062,0.023287,0.111761,0.315261,16076,16076,0
4,0.323865,0.036216,0.035917,0.037595,0.236111,0.330296,4764,4764,4
...,...,...,...,...,...,...,...,...,...
19903,0.130617,0.095281,0.063305,0.244168,0.175632,0.290997,12998,12998,5
19904,0.597983,0.070345,0.031782,0.018322,0.021660,0.259908,8377,8377,0
19905,0.056062,0.243812,0.200636,0.054803,0.048698,0.395989,2729,2729,5
19906,0.081086,0.394845,0.264938,0.059856,0.013147,0.186128,22703,22703,1


In [ ]:
from sklearn.metrics import f1_score

f1_score(OOFOut['genre_id'],OOF_preds,average="micro")

In [ ]:
pd.crosstab(OOF_preds,OOFOut['genre_id'])

In [ ]:
df = OOFOut.groupby('genre_id').size()
df = 1. / df
df = df / df.mean()

class_weights = df.values
logits_weights = np.log((class_weights).reshape((1, -1)))
probit_weights = scipy.special.softmax(logits_weights,1)

In [ ]:
probit_weights

In [ ]:
f1_score(OOFOut['genre_id'],OOFOut.iloc[:,:19].idxmax(1),average="micro")

In [ ]:
for a in [13,14,15,16]:
    print(a, OOFOut[OOFOut['genre_id'].isin([a])][a].mean(),OOFOut[~OOFOut['genre_id'].isin([a])][a].mean())

### Mode based sub

In [ ]:
kernel_type

In [ ]:
out = submission.copy().drop('genre_id',1)
for fld in range(5):
    fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fld}.csv')
    out = pd.merge(out,fname[['song_id','target']],on='song_id',how='left')
    
submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [ ]:
submission.loc[submission['song_id']==22612,'genre_id'] = 1
submission.loc[submission['song_id']==24013,'genre_id'] = 0
submission['genre_id'] = submission['genre_id'].astype(int)
submission.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/5Fold_mode.csv',index=False)

### Mode based sub

In [ ]:
out = submission.copy().drop('genre_id',1)
for fld in range(5):
    for kernels in ['resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128',
                    'resnest50d_MonoSpec_Hop512_Mels128',
                    ]:
        fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/fold_{fld}.csv')
        out = pd.merge(out,fname[['song_id','target']],on='song_id',how='left')
    
submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [ ]:
submission.loc[submission['song_id']==22612,'genre_id'] = 1
submission.loc[submission['song_id']==24013,'genre_id'] = 0
submission['genre_id'] = submission['genre_id'].astype(int)
submission.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/5Fold_mode_ens_2bestmodels.csv',index=False)

### Mode based sub: specific folds only

In [ ]:
out = submission.copy().drop('genre_id',1)
for fld in range(5):
    for kernels in ['resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128',
                    'resnest50d_MonoSpec_Hop512_Mels128',
                    ]:
        if kernels == 'resnest50d_MonoSpec_Hop512_Mels128':
            if fld in [0,1,2]:
                fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/fold_{fld}.csv')
                out = pd.merge(out,fname[['song_id','target']],on='song_id',how='left')
        else:
            fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/fold_{fld}.csv')
            out = pd.merge(out,fname[['song_id','target']],on='song_id',how='left')

submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [ ]:
submission.loc[submission['song_id']==22612,'genre_id'] = 1
submission.loc[submission['song_id']==24013,'genre_id'] = 0
submission['genre_id'] = submission['genre_id'].astype(int)
submission.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/5Fold_mode_ens_2bestmodels_bestfoldsonly.csv',index=False)

### OOF analysis

In [ ]:
OOF = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/oof_fold_{fld}_probs.npy',allow_pickle=True)

OOF = ([item for sublist in OOF for item in sublist])
OOF = pd.DataFrame(OOF)

In [ ]:
OOFOut.shape

In [ ]:
OOFOut

In [ ]:
kernel_type_upd = 'resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128' 

In [ ]:
fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type_upd}/fold_{fld}.csv')
fname

In [ ]:
preds_out = pd.DataFrame(np.zeros((5076,19)))
for fld in range(5):
    probs = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type_upd}/test_fold_{fld}_probs.npy',allow_pickle=True)
    p = pd.DataFrame([item.numpy().ravel() for sublist in probs for item in sublist])
    preds_out += p.values/5

In [ ]:
preds_out.shape

In [ ]:
preds_out['song_id'] = fname['song_id']

In [ ]:
preds_out.head()

In [ ]:
preds_out[(preds_out.iloc[:,:19].values > 0.85).sum(1)>0]

In [ ]:
PL = preds_out[(preds_out.iloc[:,:19].values > 0.85).sum(1)>0].reset_index(drop=True)

In [ ]:
PL.to_csv(r'C:\Users\Kaggle\Pog_Music_Classification\outputs\resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128\PL.csv')

In [ ]:
PL.head()

In [ ]:
np.argmax(PL.iloc[:,:-1].values,1)

### End ###